# AI-Enhanced Data Analytics & Scenario Modeling
## Project Overview

This project demonstrates an end-to-end AI-augmented analytics system for operational manufacturing data. It combines statistical analysis, Monte Carlo simulation, and LLM-powered interpretation to deliver actionable strategic insights.

---

### Project Structure

```
LLM Project/
│
├── notebooks/
│   └── 00_project_overview.ipynb          ← You are here
│
├── 01_scenario_modeling/                   ← DELIVERABLE 1
│   ├── notebooks/                          Walk-through notebook
│   ├── src/                                Modeling engine + AI analyst
│   ├── data/                               Raw data
│   └── outputs/                            Charts, CSVs
│
├── 02_insight_extraction/                  ← DELIVERABLE 2
│   ├── notebooks/                          Walk-through notebook
│   ├── src/                                Extraction framework
│   ├── data/                               Raw data
│   └── outputs/                            Charts, CSVs
│
├── 03_strategic_dashboard/                 ← DELIVERABLE 3
│   ├── notebooks/                          Component preview notebook
│   ├── src/                                Dash application
│   ├── data/                               Raw data
│   └── outputs/                            Chart exports
│
├── 04_documentation/                       ← DELIVERABLE 4
│   ├── architecture/                       System & AI architecture docs
│   ├── guides/                             Quick start, presentation guide
│   └── training/                           Extension & training materials
│
├── shared/                                 Shared code & config
│   ├── config/                             YAML settings + loader
│   ├── utils/                              LLM client, processing, plotting
│   └── data_generation/                    Synthetic data generator
│
├── main.py                                 CLI entry point
├── pyproject.toml                          Dependencies
└── README.md                               Project overview
```

---

## Quick Setup

In [ ]:
import sys
sys.path.insert(0, '..')  # project root

import importlib.util
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

from shared.data_generation.generate import generate_operational_metrics, save_data
from shared.utils.processing import run_full_processing
from shared.utils.plotting import apply_portfolio_style, save_figure, COLORS, SCENARIO_COLORS

def import_from(module_path, module_name):
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    mod = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(mod)
    return mod

pd.set_option('display.float_format', '{:.2f}'.format)
print('Setup complete.')

## Step 1: Generate & Explore Data

We generate 24 months of synthetic operational metrics for 3 manufacturing facilities, covering 10 KPIs.

In [ ]:
raw_df = generate_operational_metrics()

# Save to all deliverable data folders
for folder in ['01_scenario_modeling', '02_insight_extraction', '03_strategic_dashboard']:
    save_data(raw_df, f'../{folder}/data')

print(f'Shape: {raw_df.shape}')
print(f'Date range: {raw_df["date"].min().date()} → {raw_df["date"].max().date()}')
print(f'Facilities: {sorted(raw_df["facility"].unique())}')
print(f'Metrics ({raw_df["metric"].nunique()}): {sorted(raw_df["metric"].unique())}')
raw_df.head()

In [ ]:
# Quick overview visualization
key_metrics = ['production_output', 'quality_rate', 'defect_rate_ppm', 'on_time_delivery_pct']
fig = px.line(
    raw_df[raw_df['metric'].isin(key_metrics)],
    x='date', y='value', color='facility', facet_row='metric',
    color_discrete_map=COLORS, height=700,
)
fig = apply_portfolio_style(fig, 'Key Operational Metrics — 24 Month Overview')
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split('=')[-1].replace('_', ' ').title()))
fig.show()

## Step 2: Process Data

In [ ]:
processed = run_full_processing(raw_df)

print('Processing pipeline results:')
for name, df in processed.items():
    print(f'  {name:15s} → {df.shape[0]:>5} rows × {df.shape[1]} cols')

print(f'\nAnomalies detected: {len(processed["anomalies"])}')
print(f'Strong correlations (|r|>0.5): {len(processed["correlations"][processed["correlations"]["correlation"].abs() > 0.5])}')

## Step 3: Run Scenario Modeling

In [ ]:
modeler = import_from('../01_scenario_modeling/src/modeler.py', 'modeler')

scenarios = modeler.run_all_scenarios(raw_df)
endpoint_df = modeler.scenarios_endpoint_summary(scenarios)

print(f'Scenario projections: {len(endpoint_df)} endpoint forecasts')
print(f'Scenarios: {sorted(endpoint_df["scenario"].unique())}')
print(f'Projection horizon: {endpoint_df["projection_months"].iloc[0]} months')

# Fan chart example
fac, met = 'Plant Alpha', 'production_output'
fig = go.Figure()
for sname, proj in scenarios[fac][met].items():
    color = SCENARIO_COLORS.get(sname, '#888')
    fig.add_trace(go.Scatter(
        x=pd.concat([proj['month'], proj['month'][::-1]]),
        y=pd.concat([proj['p90'], proj['p10'][::-1]]),
        fill='toself', fillcolor=color, opacity=0.15,
        line=dict(width=0), showlegend=False,
    ))
    fig.add_trace(go.Scatter(
        x=proj['month'], y=proj['median'], mode='lines',
        name=sname.replace('_', ' ').title(), line=dict(color=color, width=2.5),
    ))
fig = apply_portfolio_style(fig, f'Scenario Projection: {met.replace("_"," ").title()} — {fac}')
fig.update_xaxes(title_text='Months Ahead')
fig.update_yaxes(title_text='Projected Value')
fig.update_layout(height=420)
fig.show()

## Step 4: AI-Powered Insight Extraction

> Requires `ANTHROPIC_API_KEY` in `.env`. Cells gracefully skip if not set.

In [ ]:
try:
    extractor = import_from('../02_insight_extraction/src/extractor.py', 'extractor')
    insights = extractor.extract_all_insights(processed)
    
    for category, text in insights.items():
        print(f'\n{"="*70}')
        print(f'  {category.upper().replace("_", " ")}')
        print(f'{"="*70}')
        print(text[:500] + '...' if len(text) > 500 else text)
except Exception as e:
    print(f'AI insights skipped: {e}')
    print('Set ANTHROPIC_API_KEY in .env to enable. All other features work without it.')

## Step 5: Launch Dashboard

The interactive dashboard combines everything above into a single strategic tool.

```bash
# From the project root:
python main.py dashboard
# Open http://localhost:8050
```

Features:
- KPI cards with MoM change indicators
- Filterable trend charts, scenario projections, anomaly plots
- Correlation heatmaps and facility radar comparison
- On-demand AI insights panel (Claude API)

---

## Deliverable Summary

| # | Deliverable | Key Files | What It Demonstrates |
|---|-------------|-----------|---------------------|
| 1 | **Scenario Modeling System** | `01_scenario_modeling/` | Monte Carlo simulation, fan charts, AI scenario interpretation |
| 2 | **Insight Extraction Framework** | `02_insight_extraction/` | 6 analysis types, prompt engineering, statistical + AI hybrid |
| 3 | **Strategic Dashboard** | `03_strategic_dashboard/` | Plotly Dash app, real-time filtering, integrated AI panel |
| 4 | **Documentation & Training** | `04_documentation/` | Architecture docs, quick start, presentation guide, extension guide |

### Individual Notebooks

Open each deliverable's notebook for a detailed walkthrough:

1. [`01_scenario_modeling/notebooks/01_scenario_modeling_walkthrough.ipynb`](../01_scenario_modeling/notebooks/01_scenario_modeling_walkthrough.ipynb)
2. [`02_insight_extraction/notebooks/02_insight_extraction_walkthrough.ipynb`](../02_insight_extraction/notebooks/02_insight_extraction_walkthrough.ipynb)
3. [`03_strategic_dashboard/notebooks/03_dashboard_walkthrough.ipynb`](../03_strategic_dashboard/notebooks/03_dashboard_walkthrough.ipynb)